In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data2img import load_data
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision.models as models
from sklearn.metrics import roc_auc_score

In [5]:
class CausalDataset(Dataset):

    def __init__(self, input_dir, target_dir=None, size=32, regression=True):
        if target_dir != None:
            self.data, self.label = load_data(
                input_dir, target_dir, size=size, regression=regression
            )
        else:
            self.data = load_data(
                input_dir, size=size, regression=regression
            )
            self.label = None
        

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X = torch.from_numpy(self.data[idx]).float()
        if self.label != None:
            y = torch.from_numpy(np.array([self.label[idx]]))
            return X, y.float()
        
        return X

In [6]:
class Net(nn.Module):
    def __init__(self, size = 32):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        if size ==32:
            self.fc1 = nn.Linear(512, 256)
        else:
            self.fc1 = nn.Linear(4608, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
size = 64
train_dataset = CausalDataset(
    'data/train_pairs.csv', 
    'data/train_target.csv', 
    size = size, regression = False,
)

test_dataset = CausalDataset(
    'data/valid_pairs.csv', 
    'data/valid_target.csv', 
    size = size, regression = False,
)

In [8]:
trainloader = DataLoader(train_dataset, batch_size=16,
                        shuffle=True, num_workers=0)
testloader = DataLoader(test_dataset, batch_size=16,
                        shuffle=True, num_workers=0)

In [9]:
net = Net(size=size)
# net = models.resnet18(pretrained = False)
# net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# net.fc = nn.Linear(512, 1)
###############################
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)
###############################
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [10]:
metrics = {
    'train_acc': [], 'train_auc':[],
    'test_acc': [], 'test_auc': [],
    'best_target': [], 'best_sub':[]
}
for epoch in range(50):  # loop over the dataset multiple times
    epoch_loss = 0
    running_loss = 0.0
    y = []
    yhat = []
    net.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        y.append(labels.detach().numpy().flatten())
        yhat.append(torch.round(torch.sigmoid(outputs.detach())).numpy().flatten())
        # print statistics
        epoch_loss += outputs.shape[0] * loss.item()
    epoch_loss = epoch_loss/len(trainloader)
    y, yhat = np.hstack(y), np.hstack(yhat)
    correct = np.sum(y==yhat)
    acc = correct/len(y)
    auc = roc_auc_score(y, yhat)
    
    net.eval()
    submission = []
    target = []
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = net(inputs)
            target.append(labels.detach().numpy().flatten())
            submission.append(torch.round(torch.sigmoid(outputs.detach())).numpy().flatten())
    submission = np.hstack(submission)
    target = np.hstack(target)
    test_correct = np.sum(target==submission)
    test_acc = test_correct/len(target)
    test_auc = roc_auc_score(target, submission)
    
    print(
        f'Epoch: {epoch + 1}, loss: {epoch_loss:.3f} '+
        f'train_acc: {acc:.3f}, train_auc: {auc:.3f} '+
        f'test_acc: {test_acc:.3f}, test_auc: {test_auc:.3f}'
    )
    metrics['train_acc'].append(acc)
    metrics['train_auc'].append(auc)
    if len(metrics['test_auc']) > 0:
        if test_auc > np.max(metrics['test_auc']):
            metrics['best_target'] = target
            metrics['best_sub'] = submission
    else:
        metrics['best_target'] = target
        metrics['best_sub'] = submission
    metrics['test_acc'].append(test_acc)
    metrics['test_auc'].append(test_auc)
print('Finished Training')

Epoch: 1, loss: 9.293 train_acc: 0.746, train_auc: 0.500 test_acc: 0.774, test_auc: 0.500
Epoch: 2, loss: 9.145 train_acc: 0.747, train_auc: 0.500 test_acc: 0.774, test_auc: 0.500
Epoch: 3, loss: 9.051 train_acc: 0.747, train_auc: 0.500 test_acc: 0.774, test_auc: 0.500
Epoch: 4, loss: 8.876 train_acc: 0.747, train_auc: 0.500 test_acc: 0.774, test_auc: 0.500
Epoch: 5, loss: 8.744 train_acc: 0.748, train_auc: 0.507 test_acc: 0.768, test_auc: 0.498
Epoch: 6, loss: 8.645 train_acc: 0.746, train_auc: 0.503 test_acc: 0.774, test_auc: 0.500
Epoch: 7, loss: 8.506 train_acc: 0.748, train_auc: 0.515 test_acc: 0.774, test_auc: 0.500
Epoch: 8, loss: 8.416 train_acc: 0.755, train_auc: 0.529 test_acc: 0.782, test_auc: 0.537
Epoch: 9, loss: 8.128 train_acc: 0.765, train_auc: 0.566 test_acc: 0.789, test_auc: 0.556
Epoch: 10, loss: 7.880 train_acc: 0.772, train_auc: 0.582 test_acc: 0.775, test_auc: 0.555
Epoch: 11, loss: 7.569 train_acc: 0.782, train_auc: 0.601 test_acc: 0.768, test_auc: 0.527
Epoch: 1

In [11]:
print(np.max(metrics['test_auc']))
print(np.max(metrics['test_acc']))

0.6207107420932957
0.7914110429447853


In [12]:
np.savez('cnn_%d.npz'%size, metrics)